<a href="https://colab.research.google.com/github/muazgulfam/AgroSense/blob/main/Crop_Disease_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ Upload your kaggle.json
from google.colab import files
uploaded = files.upload()  # Upload kaggle.json

Saving kaggle.json to kaggle.json


In [1]:
import os
if not os.path.exists('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("Drive already mounted.")

Mounted at /content/drive


In [ ]:
# ✅ Move kaggle.json to correct path
!mkdir -p /root/.config/kaggle
!cp kaggle.json /root/.config/kaggle/
!chmod 600 /root/.config/kaggle/kaggle.json

# ✅ Import and authenticate
from kaggle.api.kaggle_api_extended import KaggleApi
import os

api = KaggleApi()
api.authenticate()

# ✅ Define dataset and target path in Google Drive
KAGGLE_DATASET = "seroshkarim/cotton-leaf-disease-dataset"
TARGET_PATH = "/content/drive/MyDrive/FYP_DATASET/Crop_Dataset/Cotton_Disease_Dataset"

# ✅ Mount Google Drive (only if not already mounted)
#from google.colab import drive
#drive.mount('/content/drive')

# ✅ Make sure the directory exists
os.makedirs(TARGET_PATH, exist_ok=True)

# ✅ Download and unzip directly into Drive
api.dataset_download_files(KAGGLE_DATASET, path=TARGET_PATH, unzip=True)

print("✅ Dataset downloaded and extracted successfully to:", TARGET_PATH)

Dataset URL: https://www.kaggle.com/datasets/seroshkarim/cotton-leaf-disease-dataset
✅ Dataset downloaded and extracted successfully to: /content/drive/MyDrive/FYP_DATASET/Crop_Dataset/Cotton_Disease_Dataset


In [ ]:
!rm -r "/content/drive/MyDrive/FYP_DATASET/Crop_Dataset/Cotton_Disease_Dataset"

In [2]:
!ls "/content/drive/MyDrive/FYP_DATASET/Crop_Dataset"

Cotton_Disease_Dataset	Mango_Disease_Dataset  Tomato_Disease_Dataset
Guava_Disease_Dataset	Rice_Disease_Dataset


In [ ]:
import os
import shutil

# Dataset path
BASE_PATH = "/content/drive/MyDrive/FYP_Dataset/crop_dataset"

# Manual split configuration based on your data
split_config = {
    "Guava_Disease_Dataset": {
        "Canker": [57, 166, 55],
        "Dot": [37, 106, 35],
        "Healthy": [259, 773, 257],
        "Mummification": [55, 165, 55],
        "Rust": [57, 167, 55],
    },
    "Tomato_Disease_Dataset": {
        disease: [0, 1000, 100] for disease in [
            "Bacterial Spot", "Early Blight", "Late Blight", "Leaf mold",
            "Septoria leaf spot", "Spider mites", "Target spot",
            "Yellow Leaf Curl", "Mosaic virus", "Healthy"
        ]
    },
    "Mango_Disease_Dataset": {
        disease: [0, 500, 0] for disease in [
            "Anthracnose", "Bacterial Canker", "Cutting Weevil", "Die Black",
            "Gall Midge", "Healthy", "Powdery Mildew", "Sooty Mould"
        ]
    },
    "Cotton_Disease_Dataset": {
        "Aphids": [0, 800, 0],
        "Army Worm": [0, 800, 0],
        "Bacterial blight": [0, 800, 0],
        "Healthy": [0, 800, 0],
        "Powdery Mildew": [0, 800, 0],
        "Target Spot": [0, 788, 0]
    },
    "Rice_Disease_Dataset": {
        disease: [0, 350, 88] for disease in [
            "Bacterial Leaf Blight", "Brown Spot", "Healthy",
            "Leaf blast", "Leaf Scald", "Narrow Brown Spot"
        ]
    }
}

def safe_move(images, start, end, src_path, dst_path):
    os.makedirs(dst_path, exist_ok=True)
    for i in range(start, end):
        if i < len(images):
            shutil.move(os.path.join(src_path, images[i]), os.path.join(dst_path, images[i]))

for dataset, classes in split_config.items():
    dataset_path = os.path.join(BASE_PATH, dataset)

    # Skip already structured datasets
    if all(os.path.isdir(os.path.join(dataset_path, split)) for split in ['train', 'val', 'test']):
        print(f"✅ Already structured: {dataset}")
        continue

    for cls, (test_count, train_count, val_count) in classes.items():
        class_path = os.path.join(dataset_path, cls)
        if not os.path.exists(class_path): continue

        all_images = sorted(os.listdir(class_path))
        total_required = test_count + train_count + val_count
        if len(all_images) < total_required:
            print(f"⚠️ Not enough images in {cls}: found {len(all_images)}, expected {total_required}")
            continue

        # Move images
        safe_move(all_images, 0, test_count, class_path, os.path.join(dataset_path, "test", cls))
        safe_move(all_images, test_count, test_count+train_count, class_path, os.path.join(dataset_path, "train", cls))
        safe_move(all_images, test_count+train_count, test_count+train_count+val_count, class_path, os.path.join(dataset_path, "val", cls))

        # Remove old class folder if empty
        if os.path.exists(class_path) and not os.listdir(class_path):
            os.rmdir(class_path)

    print(f"✅ Structured: {dataset}")

print("🎉 All datasets structured successfully!")

✅ Structured: Guava_Disease_Dataset
✅ Structured: Tomato_Disease_Dataset
✅ Structured: Mango_Disease_Dataset
✅ Structured: Cotton_Disease_Dataset
✅ Structured: Rice_Disease_Dataset
🎉 All datasets structured successfully!


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras import layers, models
import os

In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ✅ Set the base directory where all datasets are stored
base_dir = "/content/drive/MyDrive/FYP_DATASET/Crop_Dataset"

# ✅ Define image generators
train_gen = ImageDataGenerator(rescale=1./255)
val_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

img_size = 224
batch_size = 32

# ✅ Loop through dataset folders
for dataset_name in os.listdir(base_dir):
    base_path = os.path.join(base_dir, dataset_name)

    train_dir = os.path.join(base_path, 'train')
    val_dir = os.path.join(base_path, 'val')
    test_dir = os.path.join(base_path, 'test')

    print(f"\n📁 Processing dataset: {dataset_name}")

    train_data = train_gen.flow_from_directory(train_dir, target_size=(img_size, img_size), batch_size=batch_size, class_mode='categorical')
    val_data = val_gen.flow_from_directory(val_dir, target_size=(img_size, img_size), batch_size=batch_size, class_mode='categorical')
    test_data = test_gen.flow_from_directory(test_dir, target_size=(img_size, img_size), batch_size=batch_size, class_mode='categorical', shuffle=False)

    print(f"✅ Loaded: {dataset_name}")



📁 Processing dataset: Guava_Disease_Dataset


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/FYP_DATASET/Crop_Dataset/Guava_Disease_Dataset/train'

In [4]:
import tensorflow as tf

dataset_path = '/content/drive/MyDrive/FYP_DATASET/Crop_Dataset/Guava_Disease_Dataset'

img_size = (224, 224)
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path + "/train",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path + "/val",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path + "/test",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)


Found 1377 files belonging to 5 classes.
Found 457 files belonging to 5 classes.
Found 465 files belonging to 5 classes.
